# Knowledge and Data: Practical Assignment 3 
## RDF Data, RDFS knowledge and inferencing 

YOUR NAME: Tidiane Khan

YOUR VUNetID: 2746251

*(If you do not provide your name and VUNetID we will not accept your submission).* 

### Learning objectives

At the end of this exercise you should be able to:

1. Access local an external data via SPARQL both from within a python programming environment and stand-alone with a GUI, such as [YASGUI](https://yasgui.triply.cc/), and this way integrate data from different sources  
2. Model your own first knowledge base, in this case an RDF Schema knowledge graph
3. Implement inference rules 

Follow this Notebook step-by-step. 

Of course, you can do the exercises in any Programming Editor of your liking. 
But you do not have to. Feel free to simply write code in the Notebook. When 
everythink is filled in and works, safe the Notebook and submit it 
as a Jupyter Notebook, i.e. with an ipynb extension. Please use as name of the 
Notebook your studentID+Assignment3.ipynb.  


We will not evaluate the programming style of your solutions. Yet we do look whether your solutions suggests an understanding, and whether they yield the correct output.

Note that all notebooks will automatically be checked for plagiarism: while similar answers can be expected, it is not allowed to directly copy the solutions from fellow students or TAs, or from the examples discussed during the lectures. Similarly, sharing your solutions with your peers is not allowed.

**IMPORTANT: Submit this notebook after finishing the assignment. It is not necessary to submit the created turtle files**

Before you start, you need to:

- **Install the *rdflib* Python package:** *pip install rdflib* (should already be installed from the previous assignment)
- **Install the *SPARQLWrapper* Python package:** *pip install SPARQLWrapper*
- **Install the free edition of the GraphDB Triplestore:** please follow this short [GraphDB tutorial](https://github.com/ucds-vu/knowledge-data-vu/blob/master/Tutorials/Preliminaries/tutorial-GraphDB.md). 

Then, add the file example-from-slides.ttl to a newly created database, say called assignment-3. 

**Note that you should have an active internet connection to run the code in this notebook. Also, if, for some external reason (ie internet and/or system issues), you cannot access the SPARQL endpoint, then report this to a TA as soon as possible!**

In [2]:
# install library
%pip install SPARQLWrapper

  Using cached SPARQLWrapper-2.0.0-py3-none-any.whl (28 kB)
  Obtaining dependency information for rdflib>=6.1.1 from https://files.pythonhosted.org/packages/d4/b0/7b7d8b5b0d01f1a0b12cc2e5038a868ef3a15825731b8a0d776cf47566c0/rdflib-7.0.0-py3-none-any.whl.metadata
  Using cached rdflib-7.0.0-py3-none-any.whl.metadata (11 kB)
  Using cached isodate-0.6.1-py2.py3-none-any.whl (41 kB)
Using cached rdflib-7.0.0-py3-none-any.whl (531 kB)
Note: you may need to restart the kernel to use updated packages.


## Task 1: (35 points) Integrate Local and External Data

You can integrate SPARQL queries into your Python code by using the *RDFLib* and *SPARQLWrapper* libraries. 

The following code accesses the DBPedia knowledge graph using its SPARQL endpoint, and returns the result of the SPARQL query requesting all the labels asserted to Amsterdam (test it!)  

In [3]:
# This code only works if you are online.
# If, for some reason, you cannot get this to work, then please contact a TA

from rdflib import Graph, RDF, RDFS, Namespace, Literal, URIRef
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?cityName
    WHERE { 
        <http://dbpedia.org/resource/Amsterdam> rdfs:label ?cityName 
    }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for result in results["results"]["bindings"]:
    print(result["cityName"]["value"])  

Amsterdam
أمستردام
Amsterdam
Amsterdam
Amsterdam
Άμστερνταμ
Amsterdamo
Ámsterdam
Amsterdam
Amstardam
Amsterdam
Amsterdam
Amsterdam
암스테르담
アムステルダム
Amsterdam
Amsterdam
Амстердам
Amesterdão
Amsterdam
Амстердам
阿姆斯特丹


For your convenience, we already wrote the following functions that might be useful to complete this task. 
In addition, we have loaded and printed the 'example-from-slides.ttl' dataset.

In [4]:
from rdflib import Graph, RDF, Namespace, Literal, URIRef
from SPARQLWrapper import SPARQLWrapper, JSON


# Loads the data from a certain file given as input in Turtle syntax into the Graph g  
# -------------------------
def load_graph(graph, filename):
    with open(filename, 'r') as f:
        graph.parse(f, format='turtle')
        

# Prints a certain graph given as input in Turtle syntax
# if your output shows byte string (ie, b'...') you must add '.decode()' to the print statements:
#    print(myGraph.serialize(format='turtle').decode())
# -------------------------
def serialize_graph(myGraph):
     print(myGraph.serialize(format='turtle'))
        

# Saves the Graph g in Turtle syntax to a certain file given as input
# -------------------------
def save_graph(myGraph, filename):
    with open(filename, 'w') as f:
        myGraph.serialize(filename, format='turtle')
        
    
# Changes the namespace of a certain URI given as input to a DBpedia URI 
# Example: transformToDBR("http://example.com/kad2020/Amsterdam") returns "http://dbpedia.org/resource/Amsterdam"
# -------------------------
def transformToDBR(uri):
    if isinstance(uri, Literal):
        # changes the literal to uppercase so that the object with the same name refers to an object and not the string
        return uri.upper()
    components = g.namespace_manager.compute_qname(uri)
    return "http://dbpedia.org/resource/%s"%(components[2])

# -------------------------

g = Graph()
load_graph(g, 'example-from-slides.ttl')
serialize_graph(g)


# Don't forget to run this cell before continuing the task.


@prefix ex: <http://example.com/kad/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

ex:Netherlands a ex:Country ;
    ex:contains ex:Ijsselmeer ;
    ex:containsCity ex:Rotterdam ;
    ex:hasCapital ex:Amsterdam ;
    ex:hasName "The Netherlands" ;
    ex:neighbours ex:Belgium .

ex:hasCapital rdfs:range ex:Capital ;
    rdfs:subPropertyOf ex:containsCity .

ex:neighbours rdfs:subPropertyOf ex:closeBy .

ex:Amsterdam a ex:Capital ;
    ex:closeBy ex:Germany .

ex:Belgium a ex:Country .

ex:EuropeanCountry rdfs:subClassOf ex:Country .

ex:Germany a ex:EuropeanCountry ;
    ex:hasCapital ex:Berlin .

ex:closeBy rdfs:domain ex:Location ;
    rdfs:range ex:Location .

ex:containsCity rdfs:domain ex:Country ;
    rdfs:range ex:City ;
    rdfs:subPropertyOf ex:contains .

ex:Capital rdfs:subClassOf ex:City .

ex:City rdfs:subClassOf ex:Location .

ex:Country rdfs:subClassOf ex:Location .




### A: Write a SPARQL query that finds all the cities in the dataset

As you cannot directly use class City, you will have to find those cities in the dataset (example-from-slides.ttl) using implicit information that can be deduced from the domain and ranges of the relations (e.g. things in a hasCapital relation are capitals and a capital is a city, etc.).

Save all the cities returned from the SPARQL query into the empty set "cities". 

In [5]:
cities = ()

cities = g.query("""
    PREFIX ex: <http://example.com/kad/> 

    SELECT DISTINCT ?city
        WHERE {
             ?s ex:hasCapital|ex:containsCity ?city.      
        }
       """)



for city in cities:
    print(city)
    



(rdflib.term.URIRef('http://example.com/kad/Amsterdam'),)
(rdflib.term.URIRef('http://example.com/kad/Berlin'),)
(rdflib.term.URIRef('http://example.com/kad/Rotterdam'),)


### B: For each city, find from DBpedia its longitude & latitude, and its number of inhabitants (if available)

Don't forget to adapt the namespace of the cities in your dataset when querying DBpedia, using the above function *transformToDBR(uri)*. Also note that namespaces should never use the *https* protocol.

The empty graph h should only contain the triples extracted from DBpedia, but added to the URIs with the 'ex' namespace. 
An example of a triple in h is the following triple: 
       
       ex:Amsterdam dbo:populationTotal "872680"^^xsd:nonNegativeInteger .

In [7]:
ex = Namespace("http://example.com/kad/")
owl = Namespace("http://www.w3.org/2002/07/owl#")
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")
dbo = Namespace("http://dbpedia.org/resource/")
geo = Namespace("http://www.w3.org/2003/01/geo/wgs84_pos#")


h = Graph()
h.bind("dbo", dbo)
h.bind("geo", geo)
h.bind("ex", ex)


# Your code here
for city in cities:
    city_dbr = []
    city_dbr.append(transformToDBR(city[0]))



qres = h.query(
    """
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
    SELECT ?pop ?lat ?long ?pop2 ?lat2 ?long2 ?pop3 ?lat3 ?long3
    WHERE {
      SERVICE <https://dbpedia.org/sparql> {
        <http://dbpedia.org/resource/Rotterdam> dbo:populationTotal ?pop ;
                                                geo:lat ?lat ;
                                                geo:long ?long.
                                                
        <http://dbpedia.org/resource/Amsterdam> dbo:populationTotal ?pop2 ;
                                                geo:lat ?lat2 ;
                                                geo:long ?long2.                
        
        <http://dbpedia.org/resource/Berlin> dbo:populationTotal ?pop3 ;
                                                geo:lat ?lat3 ;
                                                geo:long ?long3.                
        
      }
    }
    """
)

for row in qres:
    print(row)
    
    
h.add((ex.Rotterdam, dbo.populationTotal, Literal(row[0])))
h.add((ex.Rotterdam, geo.lat, Literal(row[1])))
h.add((ex.Rotterdam, geo.long, Literal(row[2])))
h.add((ex.Amsterdam, dbo.populationTotal, Literal(row[3])))
h.add((ex.Amsterdam, geo.lat, Literal(row[4])))
h.add((ex.Amsterdam, geo.long, Literal(row[5])))
h.add((ex.Berlin, dbo.populationTotal, Literal(row[6])))
h.add((ex.Berlin, geo.lat, Literal(row[7])))
h.add((ex.Berlin, geo.long, Literal(row[8])))

serialize_graph(h)


(rdflib.term.Literal('651157', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#nonNegativeInteger')), rdflib.term.Literal('51.9167', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')), rdflib.term.Literal('4.5', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')), rdflib.term.Literal('907976', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#nonNegativeInteger')), rdflib.term.Literal('52.3667', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')), rdflib.term.Literal('4.9', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')), rdflib.term.Literal('3677472', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#nonNegativeInteger')), rdflib.term.Literal('52.52', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')), rdflib.term.Literal('13.405', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')))
@prefix dbo: <http://dbpedia.org/resource/> .


### C: Save your results

- Merge the triples from example-from-slides.ttl with the information extracted from DBpedia. See the [documentation](https://rdflib.readthedocs.io/en/stable/merging.html) on how to accomplish this.
- Save all these triples into a new file 'extended-example.ttl'. **It is not necessary to submit this file**
- Print all triples in Turtle Syntax.


In [8]:
g = g + h

serialize_graph(g)



@prefix dbr: <http://dbpedia.org/resource/> .
@prefix ex: <http://example.com/kad/> .
@prefix geo1: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:Netherlands a ex:Country ;
    ex:contains ex:Ijsselmeer ;
    ex:containsCity ex:Rotterdam ;
    ex:hasCapital ex:Amsterdam ;
    ex:hasName "The Netherlands" ;
    ex:neighbours ex:Belgium .

ex:hasCapital rdfs:range ex:Capital ;
    rdfs:subPropertyOf ex:containsCity .

ex:neighbours rdfs:subPropertyOf ex:closeBy .

ex:Amsterdam a ex:Capital ;
    dbr:populationTotal "907976"^^xsd:nonNegativeInteger ;
    ex:closeBy ex:Germany ;
    geo1:lat "52.3667"^^xsd:float ;
    geo1:long "4.9"^^xsd:float .

ex:Belgium a ex:Country .

ex:Berlin dbr:populationTotal "3677472"^^xsd:nonNegativeInteger ;
    geo1:lat "52.52"^^xsd:float ;
    geo1:long "13.405"^^xsd:float .

ex:EuropeanCountry rdfs:subClassOf ex:Country .

ex:Germany a ex:EuropeanC

## Task 2: (25 points)  Implement Basic Inferencing Rules 

In the lecture we showed that the RDFS inference rules can be used to infer new knowledge. For example, infer class membership based on _rdfs:domain_ or infer relationships between subjects and objects based on _rdfs:subPropertyOf_. 

Create rules to inference class membership based on the RDF Schema language features 
*	For example: infer that an instance belongs to a class because of domain and range restrictions
*	For example: infer that an instance belongs to a (super)class because it also belongs to a subclass

We implemented the __rdfs2__ rule. You should implement the 5 following remaining rules:  

*     (rdfs2) If G contains the triples (aaa rdfs:domain xxx.) and (uuu aaa yyy.)  then infer the triple (uuu rdf:type xxx.)
*     (rdfs3) If G contains the triples (aaa rdfs:range xxx.) and (uuu aaa vvv.) then infer the triple (vvv rdf:type xxx .)
*     (rdfs5) If G contains the triples (uuu rdfs:subPropertyOf vvv.) and (vvv rdfs:subPropertyOf xxx.) then infer the triple
(uuu rdfs:subPropertyOf xxx.) 
*     (rdfs7) If G contains the triples (aaa rdfs:subPropertyOf bbb.) and (uuu aaa yyy.) then infer the triple (uuu bbb yyy) 
*     (rdfs9) If G contains the triples (uuu rdfs:subClassOf xxx.) and (vvv rdf:type uuu.) then infer the triple
 (vvv rdf:type xxx.)   -> this one was not mentioned in the lecture, but is a very important one. 
*     (rdfs11) If G contains the triples (uuu rdfs:subClassOf vvv.) and (vvv rdfs:subClassOf xxx.) then infer the triple
(uuu rdfs:subClassOf xxx.)


Run your rule reasoner on your knowledge graph. If you have implemented everything correctly, you should find exactly 17 inferences.

In [33]:
def myRDFSreasoner(myGraph):
    inferredTriples = 0
    for sbj, prd, obj in myGraph:

        # --- rdfs2 ---
        if (prd.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#domain"))):
            generator = myGraph.subject_objects(URIRef(sbj))
            for s, o in generator:
                inferredTriples += 1
                print("(rdfs 2) ", s, "rdf:type", obj)
        
        
        # --- rdfs3 ---
        if (prd.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#range"))):
            generator = myGraph.subject_objects(URIRef(sbj))
            for s, o in generator:
                inferredTriples += 1
                print("(rdfs 3) ", o, "rdf:type", obj)
        
        
        # --- rdfs5 ---
        if (prd.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#subPropertyOf"))):
            generator = myGraph.triples((URIRef(sbj), URIRef("http://www.w3.org/2000/01/rdf-schema#subPropertyOf"), None))
            for s, p, o in generator:
                inferredTriples += 1
            print("(rdfs 5) ", s, "rdf:subPropertyOf", o )
        
               
        
        # --- rdfs7 ---
        if (prd.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#subPropertyOf"))):
            generator = myGraph.triples((None, URIRef(sbj), None))
            for s, p, o in generator:
                inferredTriples += 1
            print("(rdfs 7) ", s, "ex:" + obj.split('/')[-1] , o )
        
        
        # --- rdfs9 ---
        if (prd.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#subClassOf"))):
            generator = myGraph.triples((URIRef(obj), URIRef("http://www.w3.org/2000/01/rdf-schema#subClassOf") , URIRef(sbj)))
            for s, p, o in generator:
                inferredTriples += 1
            print("(rdfs 9) ", sbj, "rdf:type" , obj)
        
        
        # --- rdfs11 ---
        if (prd.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#subClassOf"))):
            generator = myGraph.triples((URIRef(sbj), URIRef("http://www.w3.org/2000/01/rdf-schema#subClassOf") , None))
            for s, p, o in generator:
                inferredTriples += 1
            print("(rdfs 11) ", s, "rdf:subClassOf", o )
        
    print("---------------------------------")
    print("Number of inferred triples:", inferredTriples)
    print("---------------------------------")
    
myRDFSreasoner(g)  # test your reasoner



(rdfs 9)  http://example.com/kad/Country rdf:type http://example.com/kad/Location
(rdfs 11)  http://example.com/kad/Country rdf:subClassOf http://example.com/kad/Location
(rdfs 9)  http://example.com/kad/Capital rdf:type http://example.com/kad/City
(rdfs 11)  http://example.com/kad/Capital rdf:subClassOf http://example.com/kad/City
(rdfs 9)  http://example.com/kad/City rdf:type http://example.com/kad/Location
(rdfs 11)  http://example.com/kad/City rdf:subClassOf http://example.com/kad/Location
(rdfs 5)  http://example.com/kad/neighbours rdf:subPropertyOf http://example.com/kad/closeBy
(rdfs 7)  http://example.com/kad/Netherlands ex:closeBy http://example.com/kad/Belgium
(rdfs 5)  http://example.com/kad/containsCity rdf:subPropertyOf http://example.com/kad/contains
(rdfs 7)  http://example.com/kad/Netherlands ex:contains http://example.com/kad/Rotterdam
(rdfs 5)  http://example.com/kad/hasCapital rdf:subPropertyOf http://example.com/kad/containsCity
(rdfs 7)  http://example.com/kad/Germ

## Task 3: (20 points) Build your very own RDFS knowledge graph. 


Define a small RDF Schema vocabulary in Turtle. You can choose your own domain (e.g. movies, geography, sports), as long as it hasn't been used as an example during the lectures. The following rules must be respected:
*	The schema should define at least 4 classes, 4 properties, and 4 instances.
*   The properties should be used to relate the instances (i.e., object-type relations)
*	The instances should be members of at least one of the 4 defined classes
*	All resources should have an rdfs:label attribute in a suitable language.

You should use (at least) the following language features of RDF and RDFS:
* 	rdf:type (or 'a')
* 	rdfs:subClassOf
* 	rdfs:subPropertyOf
* 	rdfs:domain and rdfs:range
*	rdfs:label

Be sure to define the 'rdf:' and 'rdfs:' namespace prefixes for RDF and RDF Schema in your file (perhaps have a look at http://prefix.cc)

For creating your vocabulary you should add the axioms directly (programatically) to your Knowledge Graph as you did last week. 

Play around with the inference rules you have created in the previous task to make sure that you added some implicit knowledge, that becomes "visible" via inferencing (this will be useful for the next task). 

Finally:
- Add the knowledge you created into the RDFlib graph datastructure *myRDFSgraph*, 
- Print *myRDFSgraph* in Turtle so that we can check your "design"
- Save *myRDFSgraph* into a new file 'myRDFSgraph.ttl' (it is not necessary to submit this file)

In [62]:
myRDFSgraph = Graph()
myRDFSgraph.bind("ex", ex)
# Your code here.
ex = Namespace("http://example.com/kad/")
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")



myRDFSgraph.add((ex.Saka, rdf.type, ex.Player))
myRDFSgraph.add((ex.Saka, rdfs.label, Literal("Saka")))
myRDFSgraph.add((ex.Foden, rdf.type, ex.Player))
myRDFSgraph.add((ex.Foden, rdfs.label, Literal("Foden")))
myRDFSgraph.add((ex.Salah, rdf.type, ex.Player))
myRDFSgraph.add((ex.Salah, rdfs.label, Literal("Salah")))
myRDFSgraph.add((ex.Bruno, rdf.type, ex.Player))
myRDFSgraph.add((ex.Bruno, rdfs.label, Literal("Bruno")))

myRDFSgraph.add((ex.Arsenal, rdf.type, ex.Club))
myRDFSgraph.add((ex.Man_City, rdf.type, ex.Club))
myRDFSgraph.add((ex.Liverpool, rdf.type, ex.Club))
myRDFSgraph.add((ex.Man_Utd, rdf.type, ex.Club))

myRDFSgraph.add((ex.Arteta, rdf.type, ex.Coach))
myRDFSgraph.add((ex.Pep, rdf.type, ex.Coach))
myRDFSgraph.add((ex.Klopp, rdf.type, ex.Coach))
myRDFSgraph.add((ex.Ten_Hag, rdf.type, ex.Coach))

myRDFSgraph.add((ex.Edu, rdf.type, ex.DOF))
myRDFSgraph.add((ex.Txiki, rdf.type, ex.DOF))
myRDFSgraph.add((ex.Edwards, rdf.type, ex.DOF))
myRDFSgraph.add((ex.John, rdf.type, ex.DOF))

myRDFSgraph.add((ex.Claude, rdf.type, ex.Fan))
myRDFSgraph.add((ex.Paul, rdf.type, ex.Fan))
myRDFSgraph.add((ex.Grizz, rdf.type, ex.Fan))
myRDFSgraph.add((ex.Saeed, rdf.type, ex.Fan))

myRDFSgraph.add((ex.Saka, ex.playsFor, ex.Arsenal))
myRDFSgraph.add((ex.Foden, ex.playsFor, ex.Man_City))
myRDFSgraph.add((ex.Salah, ex.playsFor, ex.Liverpool))
myRDFSgraph.add((ex.Bruno, ex.playsFor, ex.Man_Utd))

myRDFSgraph.add((ex.Arteta, ex.coaches, ex.Arsenal))
myRDFSgraph.add((ex.Pep, ex.coaches, ex.Man_City))
myRDFSgraph.add((ex.Klopp, ex.coaches, ex.Liverpool))
myRDFSgraph.add((ex.Ten_Hag, ex.coaches, ex.Man_Utd))

myRDFSgraph.add((ex.Edu, ex.worksWith, ex.Arteta))
myRDFSgraph.add((ex.Txiki, ex.worksWith, ex.Pep))
myRDFSgraph.add((ex.Edwards, ex.worksWith, ex.Klopp))
myRDFSgraph.add((ex.John, ex.worksWith, ex.Ten_Hag))



myRDFSgraph.add((ex.Claude, rdf.supports, ex.Arsenal))
myRDFSgraph.add((ex.Paul, rdf.supports, ex.Man_City))
myRDFSgraph.add((ex.Grizz, rdf.supports, ex.Liverpool))
myRDFSgraph.add((ex.Saeed, rdf.supports, ex.Man_Utd))

myRDFSgraph.add((ex.worksWith, rdfs.subPropertyOf, ex.knows))
myRDFSgraph.add((ex.coaches, rdfs.subPropertyOf, ex.supports))
myRDFSgraph.add((ex.playsFor, rdfs.subPropertyOf, ex.supports))
myRDFSgraph.add((ex.playsFor, rdfs.subPropertyOf, ex.supports))

myRDFSgraph.add((ex.Player, rdfs.subClassOf, ex.Fan))
myRDFSgraph.add((ex.Coach, rdfs.subClassOf, ex.Fan))
myRDFSgraph.add((ex.DOF, rdfs.subClassOf, ex.Fan))

myRDFSgraph.add((ex.playsFor, rdfs.domain, ex.Player))
myRDFSgraph.add((ex.playsFor, rdfs.range, ex.Club))
myRDFSgraph.add((ex.supports, rdfs.range, ex.Club))
print("Now let's check what we can infer from your knowledge graph...")
print("The more rules you cover, the better!")
myRDFSreasoner(myRDFSgraph)



serialize_graph(myRDFSgraph)
save_graph(myRDFSgraph, "myRDFSgraph.ttl")


Now let's check what we can infer from your knowledge graph...
The more rules you cover, the better!
(rdfs 3)  http://example.com/kad/Arsenal rdf:type http://example.com/kad/Club
(rdfs 3)  http://example.com/kad/Man_City rdf:type http://example.com/kad/Club
(rdfs 3)  http://example.com/kad/Liverpool rdf:type http://example.com/kad/Club
(rdfs 3)  http://example.com/kad/Man_Utd rdf:type http://example.com/kad/Club
(rdfs 2)  http://example.com/kad/Saka rdf:type http://example.com/kad/Player
(rdfs 2)  http://example.com/kad/Foden rdf:type http://example.com/kad/Player
(rdfs 2)  http://example.com/kad/Salah rdf:type http://example.com/kad/Player
(rdfs 2)  http://example.com/kad/Bruno rdf:type http://example.com/kad/Player
(rdfs 9)  http://example.com/kad/Player rdf:type http://example.com/kad/Fan
(rdfs 11)  http://example.com/kad/Player rdf:subClassOf http://example.com/kad/Fan
(rdfs 5)  http://example.com/kad/worksWith rdf:subPropertyOf http://example.com/kad/knows
(rdfs 7)  http://example

## Task 4 (20 points) Compare local inferences with GraphDB results

Upload *myRDFSgraph.ttl* to GraphDB (check [the GraphDB tutorial](https://github.com/ucds-vu/knowledge-data-vu/blob/master/Tutorials/Preliminaries/tutorial-GraphDB.md) before starting to work with GraphDB).

Formulate two different SPARQL queries, and write a Python code that executes these queries over your GraphDB SPARQL endpoint (check example of Task 1).

**Each SPARQL query should return a _different type_ of inferred knowledge** (at least one triple that was not explicitly asserted in the graph).

Specify below next to your query (using a comment '# ...') which type of RDFS rule is the GraphDB reasoner using to infer this answer (rdfs2, rdfs3, rdfs5, rdfs7, rdfs9, rdfs11). 

In [70]:
# Get your GraphDB repository URL (setup -> repositories -> repository url) and assign it to the variable 'myEndpoint' below. 
# It should be similar to this: 

myEndpoint = "http://Tidianes-MacBook-Pro.local:7200/repositories/KnowledgeandData"  # KnowledgeAndData is the name of the repository
sparql = SPARQLWrapper(myEndpoint)

In [75]:
# Query 1 - Specify which RDFS rule are you testing: Testing rule rdfs9 i.e. subclass rule

# Check example of Task 1 on how to query remote SPARQL endpoints

sparql.setQuery("""
PREFIX ex: <http://example.com/kad/> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?player
    WHERE { 
        ?player rdf:type ex:Fan.
}

""")


sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for result in results["results"]["bindings"]:
    print(result["player"]["value"]) 

http://example.com/kad/Bruno
http://example.com/kad/Claude
http://example.com/kad/Edu
http://example.com/kad/Arteta
http://example.com/kad/Edwards
http://example.com/kad/Klopp
http://example.com/kad/Foden
http://example.com/kad/Grizz
http://example.com/kad/John
http://example.com/kad/Ten_Hag
http://example.com/kad/Paul
http://example.com/kad/Saeed
http://example.com/kad/Saka
http://example.com/kad/Salah
http://example.com/kad/Txiki
http://example.com/kad/Pep


In [78]:
# Query 2 - Specify which RDFS rule are you testing: rdfs7

# Check example of Task 1 on how to query remote SPARQL endpoints

sparql.setQuery("""

PREFIX ex: <http://example.com/kad/> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?person_a ?person_b 
    WHERE { 
        ?person_a ex:knows ?person_b
}

""")


sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for result in results["results"]["bindings"]:
    print(result["person_a"]["value"], " knows ", result["person_b"]["value"] ) 

http://example.com/kad/Edu  knows  http://example.com/kad/Arteta
http://example.com/kad/Edwards  knows  http://example.com/kad/Klopp
http://example.com/kad/John  knows  http://example.com/kad/Ten_Hag
http://example.com/kad/Txiki  knows  http://example.com/kad/Pep


## Submitting the assignment

Please submit this notebook (.ipynb) once you're finished with the assignment. It is not necessary to submit the created turtle files.